In [1]:
import json
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

In [2]:
file_path = "Counterlist-DE.txt"
data = open(file_path,'r')

In [3]:
data=[]
with open(file_path,'r') as f:    
    lines = f.readlines()[1:]
    for line in lines:
        line = line.replace('"', '')
        elements = line.strip().split("\t")
        data.append(elements)

In [4]:
columns=data[0]
data = data[1:]

In [5]:
df = pd.DataFrame(data=data, columns=columns)

In [6]:
df.head()

,total,mainPratique,id_pdc_img,today,fin,filtre,publicMessage,option1,debutPeriode,lastDay,...,photo,nom,current_year_default,moyD,lienPublic,pays,formule_site,debut,externalUrl,sig
0,285615,2,100012608,21/03/2020,null,0,,,01/01/2020,2290,...,System.Object[],Heidelberg_2 (DE),1,3570,100117680,fr,0,01/01/2013,https://www.eco-compteur.com,1
1,37851,2,100008949,21/03/2020,null,0,,,01/01/2020,428,...,System.Object[],L??rrach (DE),1,473,100117687,fr,0,01/01/2013,https://www.eco-compteur.com,1
2,92606,2,100011532,21/03/2020,null,0,,,01/01/2020,1587,...,System.Object[],Heilbronn (DE),1,1158,100117704,fr,0,01/01/2013,https://www.eco-compteur.com,1
3,64369,2,100013457,21/03/2020,null,0,,,01/01/2020,1005,...,System.Object[],Kirchheim u. Teck (DE),1,805,100117707,fr,0,01/01/2013,https://www.eco-compteur.com,1
4,127563,2,100005395,21/03/2020,null,0,,,01/01/2020,2041,...,System.Object[],Rostock (DE),1,1595,100117720,fr,0,01/01/2013,https://www.eco-compteur.com,1


In [7]:
import requests
import json
from datetime import datetime

def create_request():
  json_data={}
  for index, row in df.iterrows():
    url = 'http://data.eco-counter.com/ParcPublic/CounterData'
    start_day, start_month, start_year = row.debut.split('/')
    today_day, today_month, today_year = datetime.today().strftime('%d/%m/%Y').split('/')
    body = "idOrganisme=4586&idPdc={}&fin={}%2F{}%2F{}&debut={}%2F{}%2F{}&interval=4&pratiques=101137483%3B102137483%3B103137483%3B104137483".format(row.idPdc, today_day, today_month, today_year, start_day, start_month, start_year)

    headers = {
      "Accept": "text/plain, */*; q=0.01",
      "Accept-Encoding": "gzip, deflate",
      "Accept-Language": "en-US,en;q=0.9",
      "Connection": "keep-alive",
      "Content-Length": "115",
      "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
      "Cookie": "i18next=en_US; _ga=GA1.2.1682226698.1584790632; _gid=GA1.2.220973166.1584790632",
      "Host": "data.eco-counter.com",
      "Origin": "http://data.eco-counter.com",
      "Referer": "http://data.eco-counter.com/ParcPublic/?id=4586",
      "User-Agent": "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.80 Safari/537.36",
      "X-Requested-With": "XMLHttpRequest"
    }
    x = requests.post(url, body, headers=headers)
    loc=row['lon'],row['lat']
    location = str(loc)
    #clean response
    response_data = json.loads(x.text)
    json_data[location]=response_data
  return json_data

In [8]:
json_data = create_request()

In [9]:
with open('fahrrad_zaehler.txt', 'w') as f:
    f.write(json.dumps(json_data))